In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, SimplificationContext, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.core import DependencyManager, setDependencyManager
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
small = False

with db:
    pWorld = makeWorld(small=small)
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld, small=small)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

### Compute activity summary (20.16 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|20,151,838|21|20,151,838|GetValue
|PnLExplainReport:data|1|20,151,816|728|20,151,816|GetValue/Calc
|TradingContainer:NPV|11|20,085,814|208|1,825,983|GetValue
|TradingContainer:NPV|11|20,085,605|151|1,825,964|GetValue/Calc
|Portfolio:items|121|19,913,772|1,382|164,576|GetValue
|Portfolio:items|121|19,913,644|1,012|164,575|GetValue/Calc
|Workbook:items|1,104|18,917,734|12,959|17,135|GetValue
|Workbook:items|1,100|18,904,774|9,966|17,186|GetValue/Calc
|PnLExplainReport:cutoffs|1|10,784,425|20|10,784,425|GetValue
|PnLExplainReport:cutoffs|1|10,784,404|200|10,784,404|GetValue/Calc
|Root:Clocks|548|10,784,109|2,112,904|19,679|Context
|TradeOpenEvent|1,014|4,093,029|4,093,029|4,036|Db.Get
|TradeOpenEvent:ticket|22,286|3,974,565|134,843|178|GetValue
|TradingTicket|1,014|3,839,721|3,839,721|3,786|Db.Get
|Root:End|278|1,215,543|1,133,540|4,372|Context
|Root:Activity MarketData|255|1,191,975|1,105,004|4,674|Context
|Root:Amend Portfolio|255|1,111,562|1,027,677|4,359|Context
|Root:Start|270|1,076,786|992,139|3,988|Context
|Root:Amend MarketData|255|1,059,138|977,405|4,153|Context
|Root:Amend Trading|255|966,240|876,109|3,789|Context
|Portfolio:children|242|949,971|1,927|3,925|GetValue
|Portfolio:children|121|948,044|1,049|7,835|GetValue/Calc
|Root:Activity Portfolio|263|938,414|856,912|3,568|Context
|Root:Activity Trading|263|909,926|829,678|3,459|Context
|Root:Start breaks|255|892,534|812,054|3,500|Context
|PortfolioUpdateEvent:children|121|395,459|3,069|3,268|GetValue
|TradingBook|102|368,525|368,525|3,612|Db.Get
|Equity:NPV|15|169,267|196|11,284|GetValue
|Equity:NPV|15|169,070|119|11,271|GetValue/Calc
|MarketInterface:spot|15|159,765|163|10,651|GetValue
|MarketInterface:spot|15|159,602|123|10,640|GetValue/Calc
|ExternalRefData:state|15|149,026|166|9,935|GetValue
|ExternalRefData:state|15|148,860|122|9,924|GetValue/Calc
|RefData:state|15|148,287|159|9,885|GetValue
|RefData:state|15|148,127|121|9,875|GetValue/Calc
|_WorkItemEvent:book1|22,286|123,626|123,626|5|GetValue
|_WorkItemEvent:book2|22,286|122,241|114,883|5|GetValue
|TradeOpenEvent:quantity|22,286|111,307|111,307|4|GetValue
|Clock:cutoffs|2,480|108,248|10,731|43|GetValue
|Clock:cutoffs|20|97,615|160|4,880|GetValue/Calc
|Clock:parent|20|96,516|201|4,825|GetValue
|Clock:parent|20|96,314|193|4,815|GetValue/Calc
|_WorkItemEvent:item|11,143|88,830|82,556|7|GetValue
|TradeOpenEvent:action|11,143|84,484|84,484|7|GetValue
|TradeOpenEvent:premium|11,143|78,810|75,463|7|GetValue
|TradeOpenEvent:unitPrice|11,143|67,862|67,862|6|GetValue
|Event:amends|11,340|62,973|62,973|5|GetValue
|TradingBook:clock|2,200|61,308|23,142|27|GetValue
|PortfolioUpdateEvent|12|51,658|51,658|4,304|Db.Get
|RefDataUpdateEvent|9|40,671|40,671|4,519|Db.Get
|TradingBook:clock|1,100|38,165|13,248|34|GetValue/Calc
|TradingPortfolio|10|31,222|31,222|3,122|Db.Get
|Clock|5|16,873|16,873|3,374|Db.Get
|MarketInterface:source|15|9,629|144|641|GetValue
|Portfolio:clock|242|9,544|2,474|39|GetValue
|Portfolio:books|231|9,510|2,005|41|GetValue
|MarketInterface:source|15|9,484|138|632|GetValue/Calc
|Equity:refdata|15|8,510|180|567|GetValue
|Equity:refdata|15|8,330|129|555|GetValue/Calc
|MarketDataSource|2|8,010|8,010|4,005|Db.Get
|Portfolio:books|121|7,968|1,005|65|GetValue/Calc
|ClockEvent|2|7,534|7,534|3,767|Db.Get
|MarketInterface|2|7,493|7,493|3,746|Db.Get
|Portfolio:clock|121|7,069|1,377|58|GetValue/Calc
|ClockEvent:parent|8|6,706|118|838|GetValue
|Equity|2|6,273|6,273|3,136|Db.Get
|MarketDataSource:clock|30|4,621|281|154|GetValue
|RootClock|1|4,525|4,525|4,525|Db.Get
|MarketDataSource:clock|15|4,339|172|289|GetValue/Calc
|ForwardCashflow|1|3,346|3,346|3,346|Db.Get
|Entity:clock|40|965|404|24|GetValue
|Entity:clock|20|561|232|28|GetValue/Calc
|MarketInterface:sourceName|15|429|180|28|GetValue
|ForwardCashflow:NPV|11|363|139|33|GetValue
|RefDataUpdateEvent:data|58|316|316|5|GetValue
|RootClock:cutoffs|53|287|216|5|GetValue
|MarketInterface:sourceName|15|248|142|16|GetValue/Calc
|ForwardCashflow:NPV|11|223|92|20|GetValue/Calc
|Equity:assetName|15|119|119|7|GetValue
|RootClock:cutoffs|1|71|38|71|GetValue/Calc
|RootClock:cosmicAll|1|18|10|18|GetValue
|CosmicAll:dbState|1|14|9|14|GetValue
|PnLExplainReport:valuable|2|10|10|5|GetValue
|PnLExplainReport:ts2|2|8|8|4|GetValue
|PnLExplainReport:ts1|2|8|8|4|GetValue
|RootClock:cosmicAll|1|7|7|7|GetValue/Calc
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context0*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a simplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [5]:
class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)

    def calculated(self, node):
        if not node.isSimplified:
            return True
        ok = True
        for input in node.floatingTweakPoints():
            addFootnote(text='context simplification failure', info='%s in %s on %s in %s' % (str(node.key), node.ctx.name, str(input.key), input.ctx.name))
            ok = False
        return ok

    def simplify(self, key):
        if key.fullName() == 'RefData:state':
            obj = key.object()
            return (obj.clock().cutoffs,)
        if key.fullName() == 'Portfolio:items':
            obj = key.object()
            obj2 = obj.getObj(_tr.Clock, 'Trading')
            return (obj.clock().cutoffs, obj2.cutoffs)
        if key.fullName() == 'Workbook:items':
            obj = key.object()
            return (obj.clock().cutoffs,)
        
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        s = self.simplify(key)
        if s is not None: 
            nodes = [ getNode(bm) for bm in s ]
            values = [ bm() for bm in s ]
            k = [ (n.object(), n.key.fullName(), v) for n, v in zip(nodes, values) ]
            cKey = tuple(k)
            
            if cKey not in self.contexts:
                tweaks = dict( [ (bm, v) for bm, v in zip(s, values)])
                name = 'simp-%s' % len(self.contexts)
                ctx1 = SimplificationContext(tweaks, name)
                self.contexts[cKey] = ctx1
            ctx1 = self.contexts[cKey]
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
dm = DM1()
setDependencyManager(dm)


In [6]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

### Compute activity summary (11.85 seconds of wall clock time)

|key|value|
|-|-|
|Context|22
|Db.Get|2176
|GetValue|43372
|GetValue/Calc|1531


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|11,846,695|14|11,846,695|GetValue
|PnLExplainReport:data|1|11,846,681|791|11,846,681|GetValue/Calc
|TradingContainer:NPV|11|11,839,647|147|1,076,331|GetValue
|TradingContainer:NPV|11|11,839,499|96|1,076,318|GetValue/Calc
|Portfolio:items|61|11,658,648|706|191,125|GetValue
|Portfolio:items|55|11,658,558|435|211,973|GetValue/Calc
|Workbook:items|502|10,935,181|4,531|21,783|GetValue
|Workbook:items|300|10,930,649|2,587|36,435|GetValue/Calc
|PnLExplainReport:cutoffs|1|10,355,025|15|10,355,025|GetValue
|PnLExplainReport:cutoffs|1|10,355,010|239|10,355,010|GetValue/Calc
|Root:Clocks|59|10,354,674|40,621|175,502|Context
|simp-0|144|9,283,442|53,203|64,468|Context
|simp-1|200|8,788,960|870,760|43,944|Context
|TradeOpenEvent|1,014|4,262,358|4,262,358|4,203|Db.Get
|TradeOpenEvent:ticket|6,078|3,587,952|43,784|590|GetValue
|TradingTicket|1,014|3,544,168|3,544,168|3,495|Db.Get
|Root:Amend Trading|11|1,323,709|662|120,337|Context
|simp-8|144|1,322,815|53,334|9,186|Context
|simp-9|200|1,263,390|1,184,268|6,316|Context
|simp-3|144|939,297|57,411|6,522|Context
|simp-4|200|875,711|794,002|4,378|Context
|Portfolio:children|110|654,912|978|5,953|GetValue
|Portfolio:children|55|653,933|490|11,889|GetValue/Calc
|PortfolioUpdateEvent:children|55|386,851|2,119|7,033|GetValue
|TradingBook|102|356,624|356,624|3,496|Db.Get
|Equity:NPV|15|114,489|185|7,632|GetValue
|Equity:NPV|15|114,304|115|7,620|GetValue/Calc
|MarketInterface:spot|15|105,030|156|7,002|GetValue
|MarketInterface:spot|15|104,873|122|6,991|GetValue/Calc
|Clock:cutoffs|1,365|96,450|5,630|70|GetValue
|ExternalRefData:state|15|95,462|160|6,364|GetValue
|ExternalRefData:state|15|95,301|123|6,353|GetValue/Calc
|Clock:cutoffs|20|90,895|155|4,544|GetValue/Calc
|Clock:parent|20|89,791|215|4,489|GetValue
|Clock:parent|20|89,576|176|4,478|GetValue/Calc
|Root:Activity Portfolio|18|57,476|824|3,193|Context
|Root:Amend Portfolio|11|57,438|597|5,221|Context
|simp-7|144|56,633|52,619|393|Context
|simp-10|144|56,323|51,754|391|Context
|RefData:state|15|52,411|108|3,494|GetValue
|RefData:state|4|52,302|30|13,075|GetValue/Calc
|PortfolioUpdateEvent|12|49,154|49,154|4,096|Db.Get
|_WorkItemEvent:book2|6,078|38,256|31,552|6|GetValue
|RefDataUpdateEvent|9|36,016|36,016|4,001|Db.Get
|TradingPortfolio|10|34,811|34,811|3,481|Db.Get
|_WorkItemEvent:book1|6,078|34,079|34,079|5|GetValue
|_WorkItemEvent:item|3,039|32,578|25,226|10|GetValue
|TradingBook:clock|1,102|31,169|12,233|28|GetValue
|TradeOpenEvent:quantity|6,078|29,858|29,858|4|GetValue
|simp-5|4|26,767|7,587|6,691|Context
|TradeOpenEvent:premium|3,039|24,013|20,832|7|GetValue
|TradeOpenEvent:action|3,039|22,924|22,924|7|GetValue
|simp-2|2|21,348|4,215|10,674|Context
|Root:End|33|20,628|19,659|625|Context
|Root:Start|26|20,039|19,367|770|Context
|TradingBook:clock|800|18,936|7,360|23|GetValue/Calc
|TradeOpenEvent:unitPrice|3,039|18,420|18,420|6|GetValue
|Clock|5|18,377|18,377|3,675|Db.Get
|Event:amends|3,122|17,261|17,261|5|GetValue
|ClockEvent|2|9,022|9,022|4,511|Db.Get
|MarketInterface:source|15|8,389|149|559|GetValue
|Equity:refdata|15|8,242|190|549|GetValue
|MarketInterface:source|15|8,239|115|549|GetValue/Calc
|Equity:refdata|15|8,051|123|536|GetValue/Calc
|Equity|2|7,351|7,351|3,675|Db.Get
|MarketInterface|2|7,228|7,228|3,614|Db.Get
|MarketDataSource|2|6,993|6,993|3,496|Db.Get
|ClockEvent:parent|8|6,688|86|836|GetValue
|Portfolio:clock|171|6,396|1,407|37|GetValue
|Root:Amend MarketData|11|5,120|698|465|Context
|Portfolio:clock|66|4,989|556|75|GetValue/Calc
|simp-6|2|4,155|4,082|2,077|Context
|MarketDataSource:clock|23|4,130|226|179|GetValue
|Portfolio:books|105|3,943|909|37|GetValue
|MarketDataSource:clock|19|3,903|163|205|GetValue/Calc
|RootClock|1|3,488|3,488|3,488|Db.Get
|Portfolio:books|55|3,214|527|58|GetValue/Calc
|ForwardCashflow|1|3,181|3,181|3,181|Db.Get
|Root:Activity Trading|18|1,038|725|57|Context
|Root:Activity MarketData|11|955|729|86|Context
|Root:Start breaks|11|955|663|86|Context
|Entity:clock|40|938|399|23|GetValue
|Entity:clock|20|539|226|26|GetValue/Calc
|MarketInterface:sourceName|15|512|190|34|GetValue
|ForwardCashflow:NPV|11|357|146|32|GetValue
|MarketInterface:sourceName|15|321|241|21|GetValue/Calc
|RootClock:cutoffs|53|287|212|5|GetValue
|ForwardCashflow:NPV|11|211|86|19|GetValue/Calc
|Equity:assetName|15|115|115|7|GetValue
|RefDataUpdateEvent:data|15|80|80|5|GetValue
|RootClock:cutoffs|1|75|41|75|GetValue/Calc
|RootClock:cosmicAll|1|19|10|19|GetValue
|CosmicAll:dbState|1|15|11|15|GetValue
|PnLExplainReport:valuable|2|10|10|5|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|PnLExplainReport:ts2|2|9|9|4|GetValue
|PnLExplainReport:ts1|2|7|7|3|GetValue
|CosmicAll:dbState|1|3|3|3|GetValue/Calc

In [7]:
for k, ctx in sorted(dm.contexts.items(), key=lambda x: x[1].name):
    print
    print ctx.name
    for t in k:
        print '    ', t
    for t in ctx.tweaks:
        print '        ', t


simp-0
     (<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-03-30T09:18:37.409089,v=2017-03-30T09:18:37.409086>)
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-30T09:18:37.409089,v=2017-03-30T09:18:37.409086>)
         <Node: /Global/Clock/Portfolio.(Clock:cutoffs)() in simp-0 @4657025296>
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-0 @4657026576>

simp-1
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-30T09:18:37.409089,v=2017-03-30T09:18:37.409086>)
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-1 @4657028752>

simp-10
     (<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-03-30T09:18:37.466466,v=2017-03-30T09:18:37.466464>)
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-30T09:18:37.466466,v=2017-03-30T09:18:37.409086>)
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-10 @4920306960>
         <Node: /Global/Clock/Portfolio.(Clock:cutoffs)()